In [9]:

import pandas as pd
import numpy as np
import os

# Assuming that the csv file is loaded into a pandas DataFrame named df

df = pd.read_csv('C:\\Users\\zzzl0\\Desktop\\predicting-and-avoiding-dog-barking-behaviour\\predicting-and-avoiding-dog-barking-behaviour\\output\\model2_audio_clips\\summary.csv')

# Extracting unique dog names from 'File_Name'
df['Dog_Name'] = df['File_Name'].apply(lambda x: x.split('_')[0])
unique_dogs = df['Dog_Name'].unique()

# Create output directory if it doesn't exist
output_dir = 'C:\\Users\\zzzl0\\Desktop\\predicting-and-avoiding-dog-barking-behaviour\\predicting-and-avoiding-dog-barking-behaviour\\output\\model2_audio_clips\\augment'
os.makedirs(output_dir, exist_ok=True)

for dog in unique_dogs:
    # Filtering rows corresponding to the current dog
    df_dog = df[df['Dog_Name'] == dog]

    # Initializing new DataFrame for storing augmented data
    df_aug = pd.DataFrame(columns=['sound_class_list', 'output', 'File_Name'])

    for idx, row in df_dog.iterrows():
        sound_class_list = row[:-2].tolist()  # Converting row into list and excluding the 'File_Name' and 'Dog_Name'
        sound_class_list = [x for x in sound_class_list if not np.isnan(x)]  # Removing NaNs
        
        # Generating all continuous subsets and their corresponding class
        for i in range(len(sound_class_list) - 1):  # Excluding the last sound in each row
            for j in range(i+1, len(sound_class_list)):
                fragment = sound_class_list[i:j]
                if j == len(sound_class_list) - 1:  # Marking interval as 'Y' if fragment contains the second-last sound
                    output = 'Y'
                else:
                    output = 'N'  # Otherwise, marking it as 'N'
                    
                # Appending new row to df_aug
                new_row = pd.DataFrame([{'sound_class_list': fragment, 'output': output, 'File_Name': row['File_Name']}])
                df_aug = pd.concat([df_aug, new_row], ignore_index=True)

    # Saving the augmented dataframe to a new CSV file
    df_aug.to_csv(os.path.join(output_dir, f'{dog}_augmented_data.csv'), index=False)
